In [14]:
import zipfile as zp
import pandas as pd
import os

file_list = []

#It is preferable to use extractall for a single file. zp.zipfile extracts to buffer.
with zp.ZipFile('dataPandas.zip','r') as dataPandas:
    for file in dataPandas.namelist():
        file_list.append(file)
print(file_list)

#The DataPandas in quotes is the name of a new folder that the zip is extracted to.
zp.ZipFile('dataPandas.zip').extractall("DataPandas")
#./ is used here to find a folder named DataPandas nested within a folder named DataPandas
os.listdir('./DataPandas')

#The '.' is for working directory.
filepath = './DataPandas/DataPandas'

#Goal: Take all json person info and place it into a dataframe.





['dataPandas/3dhubs_suppliers.csv', 'dataPandas/3dhubs_suppliers_machines.csv', 'dataPandas/dataPandas/', 'dataPandas/dataPandas/3dhubs_suppliers.csv', 'dataPandas/dataPandas/3dhubs_suppliers_machines.csv', 'dataPandas/dataPandas/Sacramentorealestatetransactions.csv', 'dataPandas/dataPandas/SampleJSONfiles.json', 'dataPandas/dataPandas/SampleJSONfiles2.json', 'dataPandas/Sacramentorealestatetransactions.csv', 'dataPandas/SampleJSONfiles.json', 'dataPandas/SampleJSONfiles2.json']
